In [1]:
import numpy as np

In [2]:
# is matrix singular <=> detA == 0
def is_singular(A):
    return np.isclose(0, np.linalg.det(A))

In [3]:
# compares two vectors of floating numbers with some epsilon
def compare(A, B):
    return np.all(np.isclose(A, B))

In [4]:
# creates extended matrix in format [A|B]
def extended_matrix(A,B):
    return np.append(A, B, axis=1)

In [5]:
# implementation of basic gaussian elimination
def gauss(A, B):
    
    # check for singularity
    if is_singular(A):
        raise ValueException("Cannot solve equation for singular matrix")
    
    # create extended matrix
    U = extended_matrix(A, B)
        
    # assuming that matrix A is n by n
    n = U.shape[0]
    
    # trianguralize matrix
    for i in range(n):
        for j in range(i + 1, n):
            factor = U[j,i] / U[i,i]
            
            for k in range(n + 1):
                U[j,k] -= U[i,k] * factor
    
    # diagonalize matrix
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            factor = U[j,i] / U[i,i]
            
            for k in range(n + 1):
                U[j,k] -= U[i,k] * factor
    
    # divide to acquire ones at diagonal
    for i in range(n):
        U[i,n] /= U[i,i]
        U[i,i] /= U[i,i]
    
    # return last column as matrix X
    return U[:,n:n+1]

In [6]:
# gaussian elimination with added scaling and pivoting
def improved_gauss(A, B):
    
    # check for singularity
    if is_singular(A):
        raise ValueException("Cannot solve equation for singular matrix")
        
    # create extended matrix
    U = extended_matrix(A, B)
        
    # assuming that matrix A is n by n
    n = U.shape[0]
    
    # scale matrix
    factor = np.max(np.absolute(A), axis = 1)
    for i in range(n):
        for k in range(n+1):
            U[i,k] /= factor[i]
        
    # trianguralize matrix
    for i in range(n):
        
        # swap this row with row that has the biggest value in column i
        row = i
        for j in range(i+1, n):
            if abs(U[j,i]) > abs(U[row,i]):
                row = j
        U[[i,row]] = U[[row, i]]
        
        # same as in normal gaussian
        for j in range(i + 1, n):
            factor = U[j,i] / U[i,i]
            
            for k in range(n + 1):
                U[j,k] -= U[i,k] * factor
    
    # diagonalize matrix
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            factor = U[j,i] / U[i,i]
            
            for k in range(n + 1):
                U[j,k] -= U[i,k] * factor
    
    # divide to acquire ones at diagonal
    for i in range(n):
        U[i,n] /= U[i,i]
        U[i,i] /= U[i,i]
    
    # return last column as matrix X
    return U[:,n:n+1]

---

In [7]:
A = np.random.randn(5, 5).astype("float32")
B = np.random.randn(5, 1).astype("float32")

In [8]:
print(A)

[[ 1.1143143  -0.05188906  0.1342445  -0.03734017  0.12945895]
 [ 1.1811538  -0.27291438  0.17410973  0.26453212 -0.07622083]
 [-0.18466479  0.06602463 -0.01491731  1.3930126  -0.46863094]
 [ 1.1834173  -2.3215616   0.52494603  0.6890474  -0.02881019]
 [ 0.37875268  0.39812866  0.95619136 -0.74132305  0.77547884]]


In [9]:
print(B)

[[ 1.2029041]
 [-0.214481 ]
 [ 0.6493484]
 [ 1.816495 ]
 [-0.6529861]]


In [10]:
np.linalg.solve(A, B)

array([[ 0.5151897],
       [-0.9284308],
       [-7.9793415],
       [ 5.2997336],
       [14.28809  ]], dtype=float32)

In [11]:
gauss(A, B)

array([[ 0.5151895 ],
       [-0.92843026],
       [-7.979339  ],
       [ 5.2997336 ],
       [14.28809   ]], dtype=float32)

In [12]:
compare(np.linalg.solve(A, B), gauss(A, B))

True

---

In [13]:
A = np.array([[-0.004, -0.0001, 10], [9, 0.0004, -4], [11, -2.5, 0.005]]).astype("float32")
B = np.array([[8], [8.001], [2.5]]).astype("float32")

In [14]:
print(A)

[[-4.0e-03 -1.0e-04  1.0e+01]
 [ 9.0e+00  4.0e-04 -4.0e+00]
 [ 1.1e+01 -2.5e+00  5.0e-03]]


In [15]:
print(B)

[[8.   ]
 [8.001]
 [2.5  ]]


In [16]:
np.linalg.solve(A, B)

array([[1.2445978],
       [4.4778314],
       [0.8005426]], dtype=float32)

In [17]:
improved_gauss(A, B)

array([[1.2445978],
       [4.477831 ],
       [0.8005427]], dtype=float32)

In [18]:
gauss(A, B)

array([[1.2448579 ],
       [4.469752  ],
       [0.80054265]], dtype=float32)

In [19]:
compare(np.linalg.solve(A, B), improved_gauss(A,B))

True

In [20]:
compare(np.linalg.solve(A, B), gauss(A,B))

False

---

In [21]:
def test(n):
    A = np.random.rand(n,n)
    B = np.random.rand(n,1)
    
    test_np       = %timeit -o -r 3 -n 1 np.linalg.solve(A, B)
    test_gauss    = %timeit -o -r 3 -n 1 gauss(A, B)
    test_improved = %timeit -o -r 3 -n 1 improved_gauss(A, B)
    
    return test_np, test_gauss, test_improved

In [22]:
results = {n: tuple(map(lambda t: np.average(t.timings), test(n))) for n in range(1, 301, 10)}

The slowest run took 4.82 times longer than the fastest. This could mean that an intermediate result is being cached.
40.6 µs ± 29.3 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
158 µs ± 64.5 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
998 µs ± 671 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
The slowest run took 370.83 times longer than the fastest. This could mean that an intermediate result is being cached.
8.84 ms ± 12.3 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
2.51 ms ± 396 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
3.86 ms ± 1.55 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
The slowest run took 6.40 times longer than the fastest. This could mean that an intermediate result is being cached.
281 µs ± 178 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
12.2 ms ± 2.86 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
11.7 ms ± 1.75 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
609 µs ± 301 µs per loop (mea

In [23]:
headers = ['n','NumPy','Gaussian','Improved Gaussian']

with open('results.csv', 'w') as file:
    file.write(','.join(headers)+'\n')
    for n,res in results.items():
        file.write('{},{},{},{}\n'.format(n, *res))